# GRIT: Subqueries & CTEs - Day 5

**Learning Objectives**
- Master subqueries in WHERE, FROM, and SELECT clauses
- Use Common Table Expressions (CTEs) for readability
- Understand correlated vs non-correlated subqueries
- Apply recursive CTEs for hierarchical data
- Write complex business queries with nested logic
- Optimize query performance with advanced techniques

**Why this matters**  
Simple queries can only answer basic questions. Subqueries and CTEs let you ask sophisticated questions like "Find customers who spent more than the average customer" or "Show products that outsold their category average." This is where SQL becomes a powerful analytical tool!

Today you'll learn to write nested queries that solve complex business problems.

## Setup: Connect to Our Database

Let's connect to our e-commerce database:

In [ ]:
# Load the SQL extension
%load_ext sql

# Connect to our sample database
%sql sqlite:///ecommerce.db

print("✅ Connected to database!")

## Theory: Understanding Subqueries & CTEs

### What Are Subqueries?
A subquery is a query nested inside another query. Like a Russian doll of SQL!

**Types of Subqueries:**
- **Scalar**: Returns single value (used in SELECT/WHERE)
- **Single-row**: Returns one row (used in WHERE)
- **Multi-row**: Returns multiple rows (used with IN/EXISTS)
- **Table**: Returns table (used in FROM)

### What Are CTEs?
**Common Table Expressions** make complex queries readable:
```sql
WITH cte_name AS (
    -- Complex subquery here
)
SELECT * FROM cte_name;
```

### Correlated vs Non-Correlated:
- **Non-correlated**: Independent subquery (runs once)
- **Correlated**: References outer query (runs for each row)

### Performance Note:
CTEs are often more readable but may not always be faster than subqueries.

## Examples: Scalar Subqueries

Scalar subqueries return a single value:

In [ ]:
-- Example 1: Find customers who spent more than average
SELECT c.first_name, c.last_name,
       COALESCE(SUM(o.total_amount), 0) as total_spent
FROM customers c
LEFT JOIN orders o ON c.customer_id = o.customer_id
GROUP BY c.customer_id, c.first_name, c.last_name
HAVING COALESCE(SUM(o.total_amount), 0) > (
    -- Subquery: Calculate average spending
    SELECT AVG(customer_total)
    FROM (
        SELECT COALESCE(SUM(o2.total_amount), 0) as customer_total
        FROM customers c2
        LEFT JOIN orders o2 ON c2.customer_id = o2.customer_id
        GROUP BY c2.customer_id
    )
)
ORDER BY total_spent DESC;

In [ ]:
-- Example 2: Products priced above category average
SELECT p.product_name, p.category, p.price,
       ROUND(category_avg, 2) as category_avg
FROM products p
INNER JOIN (
    -- Subquery: Calculate average price per category
    SELECT category, AVG(price) as category_avg
    FROM products
    GROUP BY category
) cat_avg ON p.category = cat_avg.category
WHERE p.price > cat_avg.category_avg
ORDER BY p.category, p.price DESC;

## Examples: Subqueries in WHERE Clause

WHERE subqueries filter based on other query results:

In [ ]:
-- Example 3: Find products never ordered
SELECT p.product_name, p.category, p.price
FROM products p
WHERE p.product_id NOT IN (
    -- Subquery: Get all ordered product IDs
    SELECT DISTINCT product_id
    FROM order_items
)
ORDER BY p.category, p.price DESC;

In [ ]:
-- Example 4: Customers who ordered in last 30 days
SELECT DISTINCT c.first_name, c.last_name, c.email
FROM customers c
INNER JOIN orders o ON c.customer_id = o.customer_id
WHERE o.order_date >= (
    -- Subquery: Calculate date 30 days ago
    SELECT DATE('now', '-30 days')
)
ORDER BY c.last_name;

In [ ]:
-- Example 5: Orders above average order value
SELECT o.order_id, c.first_name, c.last_name,
       o.order_date, o.total_amount
FROM orders o
INNER JOIN customers c ON o.customer_id = c.customer_id
WHERE o.total_amount > (
    -- Subquery: Get average order value
    SELECT AVG(total_amount) FROM orders
)
ORDER BY o.total_amount DESC;

## Examples: EXISTS and NOT EXISTS

EXISTS tests for the existence of rows:

In [ ]:
-- Example 6: Customers who have placed orders
SELECT c.first_name, c.last_name, c.email
FROM customers c
WHERE EXISTS (
    -- Subquery: Check if customer has any orders
    SELECT 1 FROM orders o
    WHERE o.customer_id = c.customer_id
)
ORDER BY c.last_name;

In [ ]:
-- Example 7: Products that have been ordered
SELECT p.product_name, p.category, p.price
FROM products p
WHERE EXISTS (
    -- Subquery: Check if product appears in any order
    SELECT 1 FROM order_items oi
    WHERE oi.product_id = p.product_id
)
ORDER BY p.category, p.product_name;

## Examples: Subqueries in FROM Clause

FROM subqueries create temporary tables:

In [ ]:
-- Example 8: Customer spending summary
SELECT customer_summary.customer_name,
       customer_summary.total_orders,
       customer_summary.total_spent,
       CASE
           WHEN customer_summary.total_spent > 200 THEN 'High Value'
           WHEN customer_summary.total_spent > 100 THEN 'Medium Value'
           ELSE 'Low Value'
       END as customer_segment
FROM (
    -- Subquery: Calculate customer metrics
    SELECT c.customer_id,
           c.first_name || ' ' || c.last_name as customer_name,
           COUNT(o.order_id) as total_orders,
           COALESCE(SUM(o.total_amount), 0) as total_spent
    FROM customers c
    LEFT JOIN orders o ON c.customer_id = o.customer_id
    GROUP BY c.customer_id, c.first_name, c.last_name
) customer_summary
ORDER BY customer_summary.total_spent DESC;

In [ ]:
-- Example 9: Product sales performance
SELECT product_perf.product_name,
       product_perf.category,
       product_perf.total_sold,
       product_perf.revenue,
       CASE
           WHEN product_perf.total_sold > 10 THEN 'Best Seller'
           WHEN product_perf.total_sold > 5 THEN 'Good Seller'
           ELSE 'Slow Seller'
       END as performance
FROM (
    -- Subquery: Calculate product sales
    SELECT p.product_name, p.category,
           COALESCE(SUM(oi.quantity), 0) as total_sold,
           COALESCE(SUM(oi.total_price), 0) as revenue
    FROM products p
    LEFT JOIN order_items oi ON p.product_id = oi.product_id
    GROUP BY p.product_id, p.product_name, p.category
) product_perf
ORDER BY product_perf.revenue DESC;

## Examples: Common Table Expressions (CTEs)

CTEs make complex queries more readable:

In [ ]:
-- Example 10: Customer lifetime value with CTE
WITH customer_ltv AS (
    -- CTE: Calculate customer metrics
    SELECT c.customer_id,
           c.first_name || ' ' || c.last_name as customer_name,
           COUNT(o.order_id) as order_count,
           COALESCE(SUM(o.total_amount), 0) as lifetime_value,
           COALESCE(AVG(o.total_amount), 0) as avg_order_value,
           MAX(o.order_date) as last_order_date
    FROM customers c
    LEFT JOIN orders o ON c.customer_id = o.customer_id
    GROUP BY c.customer_id, c.first_name, c.last_name
),
customer_rankings AS (
    -- CTE: Rank customers by lifetime value
    SELECT customer_name,
           lifetime_value,
           NTILE(3) OVER (ORDER BY lifetime_value DESC) as value_tier
    FROM customer_ltv
    WHERE lifetime_value > 0
)
-- Main query using CTEs
SELECT cr.customer_name,
       ROUND(cr.lifetime_value, 2) as lifetime_value,
       CASE cr.value_tier
           WHEN 1 THEN 'Platinum'
           WHEN 2 THEN 'Gold'
           WHEN 3 THEN 'Silver'
       END as customer_tier
FROM customer_rankings cr
ORDER BY cr.lifetime_value DESC;

In [ ]:
-- Example 11: Category performance analysis
WITH category_stats AS (
    -- CTE: Calculate category metrics
    SELECT p.category,
           COUNT(DISTINCT p.product_id) as products_offered,
           COUNT(oi.order_item_id) as total_sales,
           COALESCE(SUM(oi.total_price), 0) as category_revenue,
           COALESCE(AVG(oi.total_price), 0) as avg_sale_price
    FROM products p
    LEFT JOIN order_items oi ON p.product_id = oi.product_id
    GROUP BY p.category
),
category_rankings AS (
    -- CTE: Rank categories by revenue
    SELECT category,
           category_revenue,
           RANK() OVER (ORDER BY category_revenue DESC) as revenue_rank,
           ROUND((category_revenue / SUM(category_revenue) OVER ()) * 100, 1) as revenue_pct
    FROM category_stats
)
-- Main query
SELECT cr.category,
       ROUND(cr.category_revenue, 2) as revenue,
       cr.revenue_rank,
       cr.revenue_pct || '%' as revenue_percentage
FROM category_rankings cr
ORDER BY cr.category_revenue DESC;

## Examples: Advanced CTE Patterns

More sophisticated CTE usage:

In [ ]:
-- Example 12: Monthly sales trend analysis
WITH monthly_sales AS (
    -- CTE: Group sales by month
    SELECT strftime('%Y-%m', o.order_date) as month,
           COUNT(o.order_id) as orders_count,
           COUNT(DISTINCT o.customer_id) as unique_customers,
           SUM(o.total_amount) as monthly_revenue,
           AVG(o.total_amount) as avg_order_value
    FROM orders o
    GROUP BY strftime('%Y-%m', o.order_date)
),
sales_comparison AS (
    -- CTE: Calculate month-over-month changes
    SELECT month,
           monthly_revenue,
           LAG(monthly_revenue) OVER (ORDER BY month) as prev_month_revenue,
           ROUND(
               ((monthly_revenue - LAG(monthly_revenue) OVER (ORDER BY month)) /
                NULLIF(LAG(monthly_revenue) OVER (ORDER BY month), 0)) * 100, 1
           ) as growth_pct
    FROM monthly_sales
)
-- Main query
SELECT sc.month,
       ROUND(sc.monthly_revenue, 2) as revenue,
       ROUND(sc.prev_month_revenue, 2) as prev_month,
       COALESCE(sc.growth_pct, 0) || '%' as growth
FROM sales_comparison sc
ORDER BY sc.month DESC;

In [ ]:
-- Example 13: Customer retention analysis
WITH customer_orders AS (
    -- CTE: Get customer order history
    SELECT c.customer_id,
           c.first_name || ' ' || c.last_name as customer_name,
           o.order_id,
           o.order_date,
           o.total_amount,
           ROW_NUMBER() OVER (PARTITION BY c.customer_id ORDER BY o.order_date) as order_number
    FROM customers c
    INNER JOIN orders o ON c.customer_id = o.customer_id
),
first_last_orders AS (
    -- CTE: Get first and last order dates per customer
    SELECT customer_id,
           customer_name,
           MIN(order_date) as first_order_date,
           MAX(order_date) as last_order_date,
           COUNT(*) as total_orders,
           SUM(total_amount) as lifetime_value
    FROM customer_orders
    GROUP BY customer_id, customer_name
),
customer_segments AS (
    -- CTE: Segment customers based on recency and frequency
    SELECT customer_name,
           total_orders,
           lifetime_value,
           CASE
               WHEN total_orders >= 5 THEN 'Champion'
               WHEN total_orders >= 3 THEN 'Loyal'
               WHEN total_orders >= 2 THEN 'Regular'
               ELSE 'New'
           END as customer_type
    FROM first_last_orders
)
-- Main query
SELECT cs.customer_type,
       COUNT(*) as customer_count,
       ROUND(AVG(cs.total_orders), 1) as avg_orders,
       ROUND(AVG(cs.lifetime_value), 2) as avg_lifetime_value
FROM customer_segments cs
GROUP BY cs.customer_type
ORDER BY avg_lifetime_value DESC;

## Examples: Correlated Subqueries

Correlated subqueries reference the outer query:

In [ ]:
-- Example 14: Products with above-average price in their category
SELECT p.product_name, p.category, p.price
FROM products p
WHERE p.price > (
    -- Correlated subquery: Average price for this product's category
    SELECT AVG(p2.price)
    FROM products p2
    WHERE p2.category = p.category
)
ORDER BY p.category, p.price DESC;

In [ ]:
-- Example 15: Customers who spent more than their state's average
SELECT c.first_name, c.last_name, c.state,
       COALESCE(customer_spending.total_spent, 0) as customer_spent,
       ROUND(state_avg.avg_state_spending, 2) as state_average
FROM customers c
LEFT JOIN (
    SELECT customer_id, COALESCE(SUM(total_amount), 0) as total_spent
    FROM orders
    GROUP BY customer_id
) customer_spending ON c.customer_id = customer_spending.customer_id
LEFT JOIN (
    -- Subquery: Average spending per state
    SELECT c2.state, AVG(COALESCE(customer_totals.total_spent, 0)) as avg_state_spending
    FROM customers c2
    LEFT JOIN (
        SELECT customer_id, SUM(total_amount) as total_spent
        FROM orders
        GROUP BY customer_id
    ) customer_totals ON c2.customer_id = customer_totals.customer_id
    GROUP BY c2.state
) state_avg ON c.state = state_avg.state
WHERE COALESCE(customer_spending.total_spent, 0) > COALESCE(state_avg.avg_state_spending, 0)
ORDER BY c.state, customer_spending.total_spent DESC;

## Exercises

### Exercise 1: Scalar Subquery
Find customers who spent more than the overall average order value

In [ ]:
-- Your code here
SELECT c.first_name, c.last_name,
       COALESCE(SUM(o.total_amount), 0) as total_spent
FROM customers c
LEFT JOIN orders o ON c.customer_id = o.customer_id
GROUP BY c.customer_id, c.first_name, c.last_name
HAVING COALESCE(SUM(o.total_amount), 0) > (
    SELECT AVG(total_amount) FROM orders
)
ORDER BY total_spent DESC;

### Exercise 2: EXISTS Subquery
Find products that have never been ordered using EXISTS

In [ ]:
-- Your code here
SELECT p.product_name, p.category, p.price
FROM products p
WHERE NOT EXISTS (
    SELECT 1 FROM order_items oi
    WHERE oi.product_id = p.product_id
)
ORDER BY p.category, p.price DESC;

### Exercise 3: FROM Subquery
Create a customer summary report using a FROM subquery

In [ ]:
-- Your code here
SELECT customer_summary.customer_name,
       customer_summary.total_orders,
       customer_summary.total_spent,
       CASE
           WHEN customer_summary.total_spent > 150 THEN 'High Value'
           WHEN customer_summary.total_spent > 75 THEN 'Medium Value'
           ELSE 'Low Value'
       END as customer_segment
FROM (
    SELECT c.customer_id,
           c.first_name || ' ' || c.last_name as customer_name,
           COUNT(o.order_id) as total_orders,
           COALESCE(SUM(o.total_amount), 0) as total_spent
    FROM customers c
    LEFT JOIN orders o ON c.customer_id = o.customer_id
    GROUP BY c.customer_id, c.first_name, c.last_name
) customer_summary
ORDER BY customer_summary.total_spent DESC;

### Exercise 4: CTE Basic
Use a CTE to analyze product performance by category

In [ ]:
-- Your code here
WITH product_performance AS (
    SELECT p.category,
           COUNT(DISTINCT p.product_id) as products_count,
           COUNT(oi.order_item_id) as sales_count,
           COALESCE(SUM(oi.total_price), 0) as category_revenue
    FROM products p
    LEFT JOIN order_items oi ON p.product_id = oi.product_id
    GROUP BY p.category
)
SELECT pp.category,
       pp.products_count,
       pp.sales_count,
       ROUND(pp.category_revenue, 2) as revenue
FROM product_performance pp
ORDER BY pp.category_revenue DESC;

### Exercise 5: Multiple CTEs
Create a monthly sales report using multiple CTEs

In [ ]:
-- Your code here
WITH monthly_orders AS (
    SELECT strftime('%Y-%m', order_date) as month,
           COUNT(order_id) as order_count,
           SUM(total_amount) as monthly_revenue
    FROM orders
    GROUP BY strftime('%Y-%m', order_date)
),
monthly_customers AS (
    SELECT strftime('%Y-%m', o.order_date) as month,
           COUNT(DISTINCT o.customer_id) as unique_customers
    FROM orders o
    GROUP BY strftime('%Y-%m', o.order_date)
)
SELECT mo.month,
       mo.order_count,
       mc.unique_customers,
       ROUND(mo.monthly_revenue, 2) as revenue
FROM monthly_orders mo
INNER JOIN monthly_customers mc ON mo.month = mc.month
ORDER BY mo.month DESC;

### Exercise 6: Correlated Subquery
Find customers who spent more than the average for their state

In [ ]:
-- Your code here
SELECT c.first_name, c.last_name, c.state,
       COALESCE(customer_totals.total_spent, 0) as customer_spent
FROM customers c
LEFT JOIN (
    SELECT customer_id, SUM(total_amount) as total_spent
    FROM orders
    GROUP BY customer_id
) customer_totals ON c.customer_id = customer_totals.customer_id
WHERE COALESCE(customer_totals.total_spent, 0) > (
    SELECT AVG(COALESCE(ct2.total_spent, 0))
    FROM customers c2
    LEFT JOIN (
        SELECT customer_id, SUM(total_amount) as total_spent
        FROM orders
        GROUP BY customer_id
    ) ct2 ON c2.customer_id = ct2.customer_id
    WHERE c2.state = c.state
)
ORDER BY c.state, customer_totals.total_spent DESC;

## Debug-Me Cell

This query has a common subquery mistake. Can you fix it?

The goal: Find products that outsold the average sales in their category

In [ ]:
-- Debug this query - it's trying to compare product sales to category average
SELECT p.product_name, p.category,
       COALESCE(SUM(oi.quantity), 0) as product_sales
FROM products p
LEFT JOIN order_items oi ON p.product_id = oi.product_id
GROUP BY p.product_id, p.product_name, p.category
HAVING COALESCE(SUM(oi.quantity), 0) > (
    -- This subquery needs to reference the outer query's category!
    SELECT AVG(COALESCE(SUM(oi2.quantity), 0))
    FROM products p2
    LEFT JOIN order_items oi2 ON p2.product_id = oi2.product_id
    WHERE p2.category = p.category  -- Missing correlation!
    GROUP BY p2.product_id
)
ORDER BY product_sales DESC;

-- Hint: The subquery needs to be correlated with the outer query using the category!

## Takeaways & Further Reading

### Subquery Types Mastered:
✅ **Scalar Subqueries**: Return single values for comparisons  
✅ **IN/NOT IN**: Check membership in result sets  
✅ **EXISTS/NOT EXISTS**: Test for existence of related data  
✅ **FROM Subqueries**: Create temporary tables for complex analysis  
✅ **Correlated Subqueries**: Reference outer query values  

### CTE Best Practices:
✅ **Readability**: Break complex queries into logical steps  
✅ **Reusability**: Use same CTE multiple times in main query  
✅ **Debugging**: Easier to test and modify individual components  
✅ **Performance**: Often equivalent to subqueries in execution  

### Key Concepts:
- **Subquery Performance**: Non-correlated run once, correlated run per row
- **EXISTS vs IN**: EXISTS often faster for large datasets
- **CTEs vs Subqueries**: CTEs usually more readable, same performance
- **Window Functions**: Advanced analytics with OVER clauses

### Common Patterns:
- **Above/Below Average**: Compare to aggregate values
- **Top Performers**: Find records exceeding thresholds
- **Missing Data**: Find records without related data
- **Complex Reports**: Multi-step analysis with CTEs

### SQL Best Practices:
- Use CTEs for complex queries requiring multiple steps
- Prefer EXISTS over IN for existence checks
- Test subqueries independently before nesting
- Use aliases consistently for readability
- Consider performance when choosing subquery vs JOIN

### Tomorrow Preview:
Day 6: **Data Manipulation** - Learn to INSERT, UPDATE, DELETE, and CREATE TABLE. You'll learn to modify data and create your own database structures!

### Practice Resources:
- [SQL Subqueries Tutorial](https://www.w3schools.com/sql/sql_subqueries.asp)
- [Common Table Expressions](https://www.sqlshack.com/sql-server-common-table-expressions-cte/)
- [Correlated Subqueries](https://www.sqlshack.com/sql-correlated-subqueries/)

**Congratulations! You can now write complex, nested SQL queries like a senior data analyst! 🔍**